In [12]:
import cv2
import torch
from ultralytics import YOLO
from ultralytics.utils.plotting import Annotator  
# ------------------ 모델 불러오기 ------------------
model = YOLO("yolov8m-pose.pt")  # 자동 다운로드

# ------------------ 이미지 불러오기 ------------------
img_path = "MY_image (5).jpg" 
frame = cv2.imread(img_path)

# 출력 창 크기 지정
resized_frame = cv2.resize(frame, (400, 300))  # 원하는 크기로 조절
cv2.imshow("Pose Estimation", resized_frame)
cv2.waitKey(0)
cv2.destroyAllWindows()

# ------------------ 추론 함수 ------------------
def predict(frame, iou=0.7, conf=0.25):
    results = model(
        source=frame,
        device="0" if torch.cuda.is_available() else "cpu",
        iou=iou,
        conf=conf,
        verbose=False
    )
    return results[0]

# ------------------ 박스 시각화 ------------------
def draw_boxes(result, frame):
    for boxes in result.boxes:
        x1, y1, x2, y2, score, cls = boxes.data.squeeze().cpu().numpy()
        cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 0, 255), 1)
    return frame

# ------------------ 키포인트 시각화 ------------------
def draw_keypoints(result, frame):
    annotator = Annotator(frame, line_width=1)
    for kps in result.keypoints:
        kps = kps.data.squeeze()
        annotator.kpts(kps)

        nkps = kps.cpu().numpy()
        for idx, (x, y, score) in enumerate(nkps):
            if score > 0.5:
                cv2.circle(frame, (int(x), int(y)), 3, (0, 0, 255), cv2.FILLED)
                cv2.putText(frame, str(idx), (int(x), int(y)), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 0, 255), 1)
    return frame

# ------------------ 실행 ------------------
result = predict(frame)
frame = draw_boxes(result, frame)
frame = draw_keypoints(result, frame)

cv2.imshow("Pose Estimation", frame)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [10]:
pip install ultralytics


Note: you may need to restart the kernel to use updated packages.
